pip install needed : tk, custom tkinter, keyboard, chemprop, rdkit, ...

In [12]:


import tkinter as tk
import customtkinter
from tkinter import Tk, Canvas, Entry, Text, Button, PhotoImage, font
from tkinter import *

from pathlib import Path

from PIL import ImageTk, Image

import keyboard

from rdkit import Chem
from rdkit.Chem import Draw

import time
from time import sleep

import webbrowser
import threading

import multiprocessing as mp

from Predictor.predictor import prediction

## Code

In [23]:
import customtkinter

# Start_gui function to create the GUI

def start_gui():
    
    ## Fonctions ##
    
    # Path pour les différents assets
    OUTPUT_PATH = Path().parent
    ASSETS_PATH = OUTPUT_PATH / Path(r"assets/frame0")
    # Céation de la fenêtre
    window = customtkinter.CTk()
    customtkinter.set_appearance_mode("#1A1A1A")
    window.title("Melting point predictor")
    window.geometry("640x480")

    # Icon
    path_icon = Path("Ressources/icon.ico")
    window.iconbitmap(path_icon)

    ##List of functions used in the program##


    # Function to setup the path of the assets
    def relative_to_assets(path) :
        return ASSETS_PATH / Path(path)
    
    # Simulate button press animation by changing the relief style temporarily
    def button_clicked_animation(button_name):
        button_name.config(relief=tk.SUNKEN, activebackground="#2A2A2A", borderwidth=0, highlightthickness=0)
        window.after(70, lambda: button_name.config(relief="flat", activebackground="#2A2A2A", borderwidth=0, highlightthickness=0))


    # Fonction pour prendre l'entrée du user, vérifier si elle est valide et afficher la prédiction du melting point
    def getEntry_and_Test(entry):
        submit_button.place_forget() 
        submit_button.config(state=tk.DISABLED) # Retirer le bouton submit
        answer_label.config(text="") # Retirer la prédiciton précédente
        make_appear_progressbar() # Afficher la barre de progression
        smiles = entry.get() # User input recovery
        try:
            mol = Chem.MolFromSmiles(smiles)
            if mol is not None:
                print_molecule(mol) # Print molecule drawing
                print_prediciton(smiles) # Print the prediction
            else:
                error_label.config(text="Invalid SMILES", fg="red", font=("Georgia", 13 * -1))
                print_empty_molecule()
        finally:
            submit_button.config(state=tk.NORMAL)
            submit_button.place(x=275, y=217)


    def print_molecule(mol):
        img = Draw.MolToImage(mol, size=(200, 140))
        img = ImageTk.PhotoImage(img)
        canvas.create_image(120, 330, anchor=NW, image=img)
        canvas.image = img

    def print_prediciton(smiles):
        answer = prediction(smiles)
        answer = round(answer, 2)
        answer_label.config(text=str(answer) + "°C", fg="white", font=("Gill Sans MT", 13 * -1))
        make_disappear_progressbar()


    def print_empty_molecule():
        smiles = ""
        mol = Chem.MolFromSmiles(smiles)
        img = Draw.MolToImage(mol, size=(200, 140))
        img = ImageTk.PhotoImage(img)
        canvas.create_image(120, 330, anchor=NW, image=img)
        canvas.image = img
        make_disappear_progressbar()


    def make_appear_progressbar():
        global progress_bar
        progress_bar = customtkinter.CTkProgressBar(window, orientation="HORIZONTAL", mode="indeterminate", height=3, width=640, fg_color="#1A1A1A", bg_color="#1A1A1A", border_color="#1A1A1A", progress_color="#305EA2", indeterminate_speed=(1))
        progress_bar.pack(pady=73)
        progress_bar.start()


    def make_disappear_progressbar():
        progress_bar.stop()
        progress_bar.pack_forget()


    #Define a callback function for the github link
    def callback(url):
        webbrowser.open_new_tab(url)


    ## Objects ##

    canvas = Canvas(
    window,
    bg = "#1A1A1A",
    height = 600,
    width = 800,
    bd = 0,
    highlightthickness = 0,
    relief = "ridge"
)
    canvas.place(x = 0, y = 0)

    image_left_container = PhotoImage(
        file=relative_to_assets("left_container.png"))
    left_container = canvas.create_image(
        220.0,
        337.0,
        image=image_left_container
    )

    image_upper_container = PhotoImage(
        file=relative_to_assets("upper_container.png"))
    upper_container = canvas.create_image(
        400.0,
        46.0,
        image=image_upper_container
    )

    image_moldraw_box = PhotoImage(
        file=relative_to_assets("moldraw_box.png"))
    moldraw_box = canvas.create_image(
        219.0,
        401.0,
        image=image_moldraw_box
    )

    image_epfl_logo = PhotoImage(
        file=relative_to_assets("epfl_logo.png"))
    epfl_logo = canvas.create_image(
        735.0,
        48.0,
        image=image_epfl_logo
    )

    image_up_right_container = PhotoImage(
        file=relative_to_assets("up_right_container.png"))
    up_right_container = canvas.create_image(
        603.0,
        249.0,
        image=image_up_right_container
    )

    image_down_right_container = PhotoImage(
        file=relative_to_assets("down_right_container.png"))
    down_right_container = canvas.create_image(
        603.0,
        423.0,
        image=image_down_right_container
    )

    image_prediction_textbox = PhotoImage(
        file=relative_to_assets("prediction_textbox.png"))
    prediction_textbox = canvas.create_image(
        603.0,
        248.0,
        image=image_prediction_textbox
    )

    canvas.create_text(
        18.0,
        24.0,
        anchor="nw",
        text="Melting point predictor\n\n\n",
        fill="#FFFFFF",
        font=("Gill Sans MT", 32 * -1)
    )

    canvas.create_text(
        86.0,
        195.0,
        anchor="nw",
        text="SMILES of the molecule",
        fill="#FFFFFF",
        font=("Gill Sans MT", 13 * -1)
    )

    canvas.create_text(
        86.0,
        308.0,
        anchor="nw",
        text="Overview of the molecule",
        fill="#FFFFFF",
        font=("Gill Sans MT", 13 * -1)
    )

    canvas.create_text(
        507.0,
        213.0,
        anchor="nw",
        text="Melting point [°C]",
        fill="#FFFFFF",
        font=("Gill Sans MT", 13 * -1)
    )

    canvas.create_text(
        465.0,
        372.0,
        anchor="nw",
        text="More about the program",
        fill="#FFFFFF",
        font=("Gill Sans MT", 15 * -1)
    )

    canvas.create_text(
        465.0,
        401.0,
        anchor="nw",
        text="This program has been developed to predict the melting \npoint of certain organic molecules. Our program \nwas trained by Mordred descriptors, more details in",
        fill="#FFFFFF",
        font=("Gill Sans MT", 12 * -1)
    )

    error_label = tk.Label(window, fg="red", bg="#2A2A2A")
    error_label.place(x=90, y=250)

    answer_label = tk.Label(window, fg="white", bg="#616161")
    answer_label.place(x=573, y=237)

    entry_left_container = PhotoImage(
        file=relative_to_assets("input_textbox.png"))
    entry_bg_1 = canvas.create_image(
        169.08280181884766,
        229.7234992980957,
        image=entry_left_container
    )
    input_textbox = Entry(
        bd=0,
        bg="#616161",
        fg="#FFFFFF",
        highlightthickness=0
    )
    input_textbox.place(
        x=93.41810750961304,
        y=216.0,
        width=151.32938861846924,
        height=25.446998596191406
    )

    button_left_container = PhotoImage(
        file=relative_to_assets("submit_button.png"))
    submit_button = Button(
        image=button_left_container,
        borderwidth=0,
        highlightthickness=0,
        activebackground="#2A2A2A",
        disabledforeground="#FFFFFF",
        command= lambda: threading.Thread(target= lambda: getEntry_and_Test(input_textbox)).start(),
        relief="flat"
    )
    submit_button.place(
        x=275.0,
        y=217.0,
        width=53.0,
        height=27.0
    )


    #Create a Label to display the link
    link = Label(window, text="https://github.com/Moglul/Ppchem-MP-predictor",font=('Gill Sans MT', 12 * -1, UNDERLINE), fg="#FFFFFF", bg ="#2A2A2A", cursor="hand2")
    link.place(x=462, y=452)
    link.bind("<Button-1>", lambda e:
    callback("https://github.com/Moglul/Ppchem-MP-predictor"))

    # If the enter button is pressed, the button submit is pressed
    input_textbox.bind('<Return>', lambda x: submit_button.invoke() and button_clicked_animation(submit_button))


    window.mainloop()

if __name__ == "__main__":
    start_gui()